# Création de nouveaux sous-titres segmentés en phrase

Ce notebook permet de créer de nouveaux fichiers de sous-titres à l'aide de deux fichiers : 
1. Un fichier contenant les phrases (sous titres)
2. Un fichier de sous-titres, sous segmenté pour que chaque sous-titre correspondent à une phrase ou à un morceau de phrase (et non deux phrases dans un même sous-titre)

In [1]:
import os
import re
import module_traitement as m 

In [162]:
# Pour Matignon - LSF 
pre_seg_file = "../data/new_segmentation_cr/"
sentence_file = "../data/sentence_matignon/"
resultat = m.lister_fichiers_with_path(pre_seg_file)
path_sentence = m.lister_fichiers_with_path(sentence_file)
resultat_output = m.lister_fichiers(pre_seg_file)
sentences_only = m.lister_fichiers(sentence_file)
output_folder = "../data/new_cr_sub/"

In [70]:
# Pour Mediapi : 
pre_seg_file = "../data/new_segmentation_mediapi/"
sentence_file = "../data/sentence_mediapi/"
resultat = m.lister_fichiers_with_path(pre_seg_file)
path_sentence = m.lister_fichiers_with_path(sentence_file)
resultat_output = m.lister_fichiers(pre_seg_file)
sentences_only = m.lister_fichiers(sentence_file)
output_folder = "../data/new_mediapi_sub/"

In [85]:
# TEST Pour Matignon - LSF 
pre_seg_file = "test_new_seg_cr/"
sentence_file = "try/"
resultat = m.lister_fichiers_with_path(pre_seg_file)
path_sentence = m.lister_fichiers_with_path(sentence_file)
resultat_output = m.lister_fichiers(pre_seg_file)
sentences_only = m.lister_fichiers(sentence_file)
output_folder = "test_sent_seg_cr/"

In [2]:
# TEST Mediapi
pre_seg_file = "test_new_seg_mediapi/"
sentence_file = "test_sent_file_mediapi/"
resultat = m.lister_fichiers_with_path(pre_seg_file)
path_sentence = m.lister_fichiers_with_path(sentence_file)
resultat_output = m.lister_fichiers(pre_seg_file)
sentences_only = m.lister_fichiers(sentence_file)
output_folder = "test_sent_seg_mediapi/"

In [3]:
import module_traitement as m 

In [4]:
# Création de nouveaux fichiers de sous-titres
for sub,sub_name,phr,phr_name in zip(resultat,resultat_output,path_sentence,sentences_only):
    print(f"TRAITEMENT : {sub_name} && {phr_name}")
    print(f"TRAITEMENT : {sub} && {phr}")
    dict_sub = m.get_dict_vtt(sub)
    sentences = m.get_sentences(phr)
    keys = list(dict_sub.keys())
    i = 0
    j = 0
    mm = 0
    new_dict = {}
    content = ""
    while i < len(keys) and j < len(sentences):
        #print(mm)
        key = keys[i]
        value = dict_sub[key]
        unite = value['text'].strip()
        sent = sentences[j].strip()
        #print(f"'{unite} --> '{sent}'")
        # Je regarde si l'unité est dans la phrase. Si elle est dans la phrase, j'ajoute l'information start, et tant que les unités sont dans la même phrase je concatène pour ajouter l'information du texte
        if unite in sent:
            #print(f"'{unite} --> '{sent}'")
            if mm not in new_dict:
                new_dict[mm] = {"start": value['start']}
            if 'text' not in new_dict[mm]:
                new_dict[mm]["text"] = unite.strip()
            else:
                new_dict[mm]['text'] += f" {unite.strip()}" #c'est ici que l'espace en trop se créer parfois. Compliqué à corriger
            i = i +1

            if i == len(keys):
                new_dict[mm]['end'] = value['end']
        else:
            old_key = keys[i - 1]
            if mm in new_dict and 'end' not in new_dict[mm]:
                new_dict[mm]['end'] = dict_sub[old_key]['end']
                #print(new_dict)
                mm += 1
            j = j +1
    m.create_vtt_file(new_dict,f"{output_folder}/{sub_name}")

TRAITEMENT : 0004d291e4.vtt && 0004d291e4.vtt
TRAITEMENT : test_new_seg_mediapi/0004d291e4.vtt && test_sent_file_mediapi/0004d291e4.vtt
TRAITEMENT : 0007d118a9.vtt && 0007d118a9.vtt
TRAITEMENT : test_new_seg_mediapi/0007d118a9.vtt && test_sent_file_mediapi/0007d118a9.vtt
TRAITEMENT : 00099d279c.vtt && 00099d279c.vtt
TRAITEMENT : test_new_seg_mediapi/00099d279c.vtt && test_sent_file_mediapi/00099d279c.vtt
TRAITEMENT : 000ad4adb5.vtt && 000ad4adb5.vtt
TRAITEMENT : test_new_seg_mediapi/000ad4adb5.vtt && test_sent_file_mediapi/000ad4adb5.vtt
TRAITEMENT : 0026226fb1.vtt && 0026226fb1.vtt
TRAITEMENT : test_new_seg_mediapi/0026226fb1.vtt && test_sent_file_mediapi/0026226fb1.vtt
TRAITEMENT : 005971b87d.vtt && 005971b87d.vtt
TRAITEMENT : test_new_seg_mediapi/005971b87d.vtt && test_sent_file_mediapi/005971b87d.vtt
TRAITEMENT : 005c1d83b5.vtt && 005c1d83b5.vtt
TRAITEMENT : test_new_seg_mediapi/005c1d83b5.vtt && test_sent_file_mediapi/005c1d83b5.vtt
TRAITEMENT : 00613c92e3.vtt && 00613c92e3.vtt
TR

## Vérifications manuelles

On compare le fichier de phrases avec les nouveaux sous-titres, pour vérifier si on a la même chose. 

S'il y a une différence, on note le fichier et l'endroit, ce qui permet de faire une correction.

On corriger : 
- soit le fichier de phrases (si la phrases a été mal découpé par le modèle de langue)
- soit le fichier sur-segmenté, s'il y a eu un problème de segmentation

In [5]:
path_sentence = m.lister_fichiers_with_path("test_sent_file_mediapi/") # vérifier qu'on charge le bon fichier !
output_folder = m.lister_fichiers_with_path("test_sent_seg_mediapi")
output_folder = sorted(output_folder)
print(output_folder)

['test_sent_seg_mediapi/0004d291e4.vtt', 'test_sent_seg_mediapi/0007d118a9.vtt', 'test_sent_seg_mediapi/00099d279c.vtt', 'test_sent_seg_mediapi/000ad4adb5.vtt', 'test_sent_seg_mediapi/0026226fb1.vtt', 'test_sent_seg_mediapi/005971b87d.vtt', 'test_sent_seg_mediapi/005c1d83b5.vtt', 'test_sent_seg_mediapi/00613c92e3.vtt', 'test_sent_seg_mediapi/006c967985.vtt', 'test_sent_seg_mediapi/007e93fe10.vtt', 'test_sent_seg_mediapi/0083bddf4e.vtt', 'test_sent_seg_mediapi/008420cac0.vtt', 'test_sent_seg_mediapi/00961862b7.vtt', 'test_sent_seg_mediapi/00d88dbb91.vtt', 'test_sent_seg_mediapi/00ef761570.vtt', 'test_sent_seg_mediapi/0115c2b09b.vtt', 'test_sent_seg_mediapi/0126d5ec9a.vtt', 'test_sent_seg_mediapi/012af290a0.vtt', 'test_sent_seg_mediapi/01396e52f9.vtt', 'test_sent_seg_mediapi/013a4d2e94.vtt', 'test_sent_seg_mediapi/01438658d4.vtt', 'test_sent_seg_mediapi/014413bb15.vtt', 'test_sent_seg_mediapi/014465666a.vtt', 'test_sent_seg_mediapi/0152d6f219.vtt', 'test_sent_seg_mediapi/015d410c82.vtt',

In [6]:
path_sentence = sorted(path_sentence)
print(path_sentence)

['test_sent_file_mediapi/0004d291e4.vtt', 'test_sent_file_mediapi/0007d118a9.vtt', 'test_sent_file_mediapi/00099d279c.vtt', 'test_sent_file_mediapi/000ad4adb5.vtt', 'test_sent_file_mediapi/0026226fb1.vtt', 'test_sent_file_mediapi/005971b87d.vtt', 'test_sent_file_mediapi/005c1d83b5.vtt', 'test_sent_file_mediapi/00613c92e3.vtt', 'test_sent_file_mediapi/006c967985.vtt', 'test_sent_file_mediapi/007e93fe10.vtt', 'test_sent_file_mediapi/0083bddf4e.vtt', 'test_sent_file_mediapi/008420cac0.vtt', 'test_sent_file_mediapi/00961862b7.vtt', 'test_sent_file_mediapi/00d88dbb91.vtt', 'test_sent_file_mediapi/00ef761570.vtt', 'test_sent_file_mediapi/0115c2b09b.vtt', 'test_sent_file_mediapi/0126d5ec9a.vtt', 'test_sent_file_mediapi/012af290a0.vtt', 'test_sent_file_mediapi/01396e52f9.vtt', 'test_sent_file_mediapi/013a4d2e94.vtt', 'test_sent_file_mediapi/01438658d4.vtt', 'test_sent_file_mediapi/014413bb15.vtt', 'test_sent_file_mediapi/014465666a.vtt', 'test_sent_file_mediapi/0152d6f219.vtt', 'test_sent_file

In [7]:
def supprimer_derniere_chaine_vide(liste):
    if liste and isinstance(liste[-1], str) and not liste[-1]:
        del liste[-1]
    return liste

# Exemple d'utilisation
ma_liste = [1, 2, "abc", ""]
print("Avant : ", ma_liste)

ma_liste = supprimer_derniere_chaine_vide(ma_liste)

print("Après : ", ma_liste)


Avant :  [1, 2, 'abc', '']
Après :  [1, 2, 'abc']


In [8]:
nb_file = 0
name_file = []
good_file = []
for sub_file, sent_file in zip(output_folder,path_sentence):
    sub_dict = m.get_dict_vtt(sub_file)
    sentence = m.get_sentences(sent_file)
    new_sent = []
    for k,v in sub_dict.items():
        for kk,vv in v.items():
            if kk == "text":
                if "(... )" in vv:
                    vv = vv.replace("(... )","(...)")
                new_sent.append(vv)
    new_sent = supprimer_derniere_chaine_vide(new_sent)
    differences = m.comparer_listes(new_sent, sentence)
    if differences:
        print(f"TRAITEMENT : {sub_file} && {sent_file}")
        for position, element_a, element_b in differences:
            nb_file = nb_file +1
            print(f"Différence à la position {position}: {element_a} vs {element_b}")
            name_file.append(sub_file)
            break
    else:
        good_file.append(sub_file)

print(f"Nombres de fichiers contenant une erreur : {nb_file}")

TRAITEMENT : test_sent_seg_mediapi/01d57cb44a.vtt && test_sent_file_mediapi/01d57cb44a.vtt
Différence à la position 92: Ce S'News est terminé, notre prochain rendez vous est au mois de décembre ! Ce S'News est terminé, notre prochain rendez vous est au mois de décembre ! vs Ce S'News est terminé, notre prochain rendez vous est au mois de décembre !
TRAITEMENT : test_sent_seg_mediapi/024e1e4beb.vtt && test_sent_file_mediapi/024e1e4beb.vtt
Différence à la position 35: J'aimerai également une amélioration de la qualité de l'éducation dans les écoles... ... vs J'aimerai également une amélioration de la qualité de l'éducation dans les écoles...
TRAITEMENT : test_sent_seg_mediapi/02fd98eec7.vtt && test_sent_file_mediapi/02fd98eec7.vtt
Différence à la position 31: ... Je ne saurais vous dire si c'est obligatoire ou non, je ne le sais pas ! vs ...Je ne saurais vous dire si c'est obligatoire ou non, je ne le sais pas !
TRAITEMENT : test_sent_seg_mediapi/03a663d287.vtt && test_sent_file_mediapi/

TRAITEMENT : test_sent_seg_mediapi/13d32c0f21.vtt && test_sent_file_mediapi/13d32c0f21.vtt
Différence à la position 35: .. sans aller au fond des choses. vs ..sans aller au fond des choses.
TRAITEMENT : test_sent_seg_mediapi/14cf31e15f.vtt && test_sent_file_mediapi/14cf31e15f.vtt
Différence à la position 4: Et... ... vs Et...
TRAITEMENT : test_sent_seg_mediapi/17f9dfabed.vtt && test_sent_file_mediapi/17f9dfabed.vtt
Différence à la position 4: Ça commence le jour de l'Épiphanie, la fête des rois mages — tu sais, le jour de la galette ! — vs Ça commence le jour de l'Épiphanie, la fête des rois mages — tu sais, le jour de la galette !
TRAITEMENT : test_sent_seg_mediapi/1992ebbd09.vtt && test_sent_file_mediapi/1992ebbd09.vtt
Différence à la position 62: Marre ! Marre ! vs Marre !
TRAITEMENT : test_sent_seg_mediapi/1afbd17607.vtt && test_sent_file_mediapi/1afbd17607.vtt
Différence à la position 4: None vs J'ai réussi le B.A.F.A. depuis 1an environ..
TRAITEMENT : test_sent_seg_mediapi/1f4bcc

In [9]:
len(name_file)

268

In [19]:
# Si besoin de déplacer des fichiers 

# Liste des chemins des fichiers à déplacer
chemins_des_fichiers = good_file

# Dossier de destination
dossier_de_destination = "gold_sent_seg_mediapi/"

# Appel de la fonction de déplacement des fichiers
m.deplacer_fichiers(chemins_des_fichiers, dossier_de_destination)


Le fichier '0004d291e4.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '0007d118a9.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '00099d279c.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '000ad4adb5.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '0026226fb1.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '005971b87d.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '005c1d83b5.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '00613c92e3.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '006c967985.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '007e93fe10.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '0083bddf4e.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '008420cac0.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '00961862b7.vtt' a été déplacé vers 'gold_sent_seg_mediapi/'.
Le fichier '00d88dbb91.vtt' a été déplacé vers 'gol

In [12]:
file_with_path = m.lister_fichiers_with_path("test_sent_seg_mediapi")
folder = m.lister_fichiers("test_sent_seg_mediapi")
output_seg = ("try_seg_end/")

In [15]:
import re

import re
from itertools import groupby

import difflib

def detecter_doublons(texte):
    tokens = texte.split()
    doublons = [token for token, group in groupby(tokens) if len(list(group)) > 1]
    return doublons

texte = "Cuire. Cuire. Cuire"
doublons = detecter_doublons(texte)
print(doublons)


['Cuire.']
